*We'll start off by importing everything we need*

In [ ]:
import os
import math
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
# %matplotlib tk

In [ ]:
def Jsolve(A, b, tol = 1e-8, max_iters = 1000, callback = None):
    """Solve a linear system Ax = b for x by the Jacobi iterative method.
    Parameters: 
      A: the matrix.
      b: the right-hand side vector.
      tol = 1e-8: the relative residual at which to stop iterating.
      max_iters = 1000: the maximum number of iterations to do. 
      callback = None: a user function to call at every iteration. 
        The callback function has arguments 'x', 'iteration', and 'residual'
    Outputs (in order):
      x: the computed solution
      rel_res: list of relative residual norms at each iteration.
        The number of iterations actually done is len(rel_res) - 1
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"

    # Split A into diagonal D plus off-diagonal C
    d = A.diagonal()         # diagonal elements of A as a vector
    C = A.copy()             # copy of A ...
    C.setdiag(np.zeros(n))   # ... without the diagonal
    
    # Initial guess: x = 0
    x = np.zeros(n)

    # Vector of relative residuals
    # Relative residual is norm(residual)/norm(b)
    # Intitial residual is b - Ax for x=0, or b
    rel_res = [1.0]
        
    # Call user function if specified
    if callback is not None:
        callback(x = x, iteration = 0, residual = 1)

    # Iterate
    for k in range(1, max_iters+1):
        # New x
        x = (b - C @ x) / d

        # Record relative residual
        this_rel_res = npla.norm(b - A @ x) / npla.norm(b)
        rel_res.append(this_rel_res)
                
        # Call user function if specified
        if callback is not None:
            callback(x = x, iteration = k, residual = this_rel_res)
                        
        # Stop if within tolerance    
        if this_rel_res <= tol:
            break
            
    return (x, rel_res)

In [ ]:
def CGsolve(A, b, tol = 1e-8, max_iters = 1000, callback = None):
    """Solve a linear system Ax = b for x by the conjugate gradient iterative method.
    Parameters: 
      A: the matrix.
      b: the right-hand side vector.
      tol = 1e-8: the relative residual at which to stop iterating.
      max_iters = 1000: the maximum number of iterations to do. 
      callback = None: a user function to call at every iteration, with one argument x
    Outputs (in order):
      x: the computed solution
      rel_res: list of relative residual norms at each iteration.
        The number of iterations actually done is len(rel_res) - 1
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"

    # Initial guess: x = 0
    x = np.zeros(n)
    
    # Initial residual: r = b - A@0 = b
    r = b
 
    # Initial step is in direction of residual.
    d = r

    # Squared norm of residual
    rtr = r.T @ r
    
    # Vector of relative residuals
    # Relative residual is norm(residual)/norm(b)
    # Intitial residual is b - Ax for x=0, or b
    rel_res = [1.0]
     
    # Call user function if specified
    if callback is not None:
        callback(x = x, iteration = 0, residual = 1)

    # Iterate
    for k in range(1, max_iters+1):
        Ad = A @ d
        alpha = rtr / (d.T @ Ad)  # Length of step
        x = x + alpha * d         # Update x to new x
        r = r - alpha * Ad        # Update r to new residual
        rtrold = rtr 
        rtr = r.T @ r
        beta = rtr / rtrold    
        d = r + beta * d          # Update d to new step direction
                   
        # Record relative residual
        this_rel_res = npla.norm(b - A @ x) / npla.norm(b)
        rel_res.append(this_rel_res)
                
        # Call user function if specified
        if callback is not None:
            callback(x = x, iteration = k, residual = this_rel_res)
                        
        # Stop if within tolerance    
        if this_rel_res <= tol:
            break
            
    return (x, rel_res)

# Re: Jacobi's Method

In [ ]:
A = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
b = np.array([3, 9, -6])
print("A:\n", A)
print("\nb:\n", b)
x = npla.solve(A, b)
print("\nx:\n", x)

In [ ]:
#Run it using Jacobi
# IMPORTANT NOTE: Jsolve() requires A to be a sparse matrix
A = sparse.csr_matrix(A)

print("x and iterated residuals: \n", Jsolve(A, b))

# To see the last residual:
print("\nLast residual: ", Jsolve(A,b)[1][-1])

# Jacobi does not always converge

In [ ]:
A = np.array([[1,2],[3,4]])
b = np.array([3,7])
print("A:\n", A)
print("\nb:\n", b)
x = npla.solve(A, b)
print("\nx:\n", x)

In [ ]:
d = A.diagonal()
D = np.diag(d)
C = A - D
x = np.zeros(2)
# Check spectral radius
m = npla.pinv(D)@C
evs = npla.eig(m)[0]
if max(evs) < 1:
    print("Spectral radius < 1. Will converge.")
else:
    print("Spectral radius >= 1. Will not converge.")

In [ ]:
#Run it using Jacobi - note, Jsolve() requires A to be a sparse matrix
A = sparse.csr_matrix(A)

print("x and iterated residuals: \n", Jsolve(A, b))

# To see the last residual:
print("\nLast residual: ", Jsolve(A,b)[1][-1])

# CG Method

In [ ]:
A = np.array([[4, -1, -1], [-2, 6, 1], [-1, 1, 7]])
b = np.array([3, 9, -6])
print("A:\n", A)
print("\nb:\n", b)
x = npla.solve(A, b)
print("\nx:\n", x)

In [ ]:
CGsolve(A, b)

In [ ]:
A = np.array([[1,2],[3,4]])
b = np.array([3,7])
x = npla.solve(A, b)
print("\nx:\n", x)
CGsolve(A, b)

## Let's put in an SPD matrix for A

In [ ]:
# Using a SPD Matrix for A
A = np.array([[2,1],[1,1]])
b = np.array([1,1])
x = npla.solve(A, b)
print("\nx:\n", x)
CGsolve(A, b)

In [ ]:
# Using another SPD Matrix for A
A = np.array([[4, 1, -1], [1, 2, 1], [-1, 1, 2]])
b = 10*np.random.rand(3)
x = npla.solve(A, b)
print("\nx:\n", x)
CGsolve(A, b)

## Let's do the temperature problem with Jacobi and CG!

In [ ]:
def make_A(k):
    """Create the matrix for the temperature problem on a k-by-k grid.
    Parameters: 
      k: number of grid points in each dimension.
    Outputs:
      A: the sparse k**2-by-k**2 matrix representing the finite difference approximation to Poisson's equation.
    """
    # First make a list with one triple (row, column, value) for each nonzero element of A
    triples = []
    for i in range(k):
        for j in range(k):
            # what row of the matrix is grid point (i,j)?
            row = j + i*k
            # the diagonal element in this row
            triples.append((row, row, 4.0))
            # connect to left grid neighbor
            if j > 0:
                triples.append((row, row - 1, -1.0))
            # ... right neighbor
            if j < k - 1:
                triples.append((row, row + 1, -1.0))
            # ... neighbor above
            if i > 0:
                triples.append((row, row - k, -1.0))
            # ... neighbor below
            if i < k - 1:
                triples.append((row, row + k, -1.0))
    
    # Finally convert the list of triples to a scipy sparse matrix
    ndim = k*k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values, (rownum, colnum)), shape = (ndim, ndim))
    
    return A 

In [ ]:
def make_b(k, top = 0, bottom = 0, left = 0, right = 0):
    """Create the right-hand side for the temperature problem on a k-by-k grid.
    Parameters: 
      k: number of grid points in each dimension.
      top: list of k values for top boundary (optional, defaults to 0)
      bottom: list of k values for bottom boundary (optional, defaults to 0)
      left: list of k values for top boundary (optional, defaults to 0)
      right: list of k values for top boundary (optional, defaults to 0)
    Outputs:
      b: the k**2 element vector (as a numpy array) for the rhs of the Poisson equation with given boundary conditions
    """
    # Start with a vector of zeros
    ndim = k*k
    b = np.zeros(shape = ndim)
    
    # Fill in the four boundaries as appropriate
    b[0        : k       ] += top
    b[ndim - k : ndim    ] += bottom
    b[0        : ndim : k] += left
    b[k-1      : ndim : k] += right
    
    return b

In [ ]:
def radiator(k, width = .3, temperature = 100.):
    """Create one wall with a radiator
    Parameters: 
      k: number of grid points in each dimension; length of the wall.
      width: width of the radiator as a fraction of length of the wall (defaults to 0.2)
      temperature: temperature of the radiator (defaults to 100)
    Outputs:
      wall: the k element vector (as a numpy array) for the boundary conditions at the wall
    """
    rad_start = int(k * (0.5 - width/2))
    rad_end = int(k * (0.5 + width/2))
    wall = np.zeros(k)
    wall[rad_start : rad_end] = temperature
    
    return wall

In [ ]:
k = 100
rad_wall = radiator(k)
A = make_A(k)
b = make_b(k, right = rad_wall)

t = spla.spsolve(A,b)
print("t:", t)
print("size:", t.size)

In [ ]:
#Set it up for plotting and then plot 3D projection
T = t.reshape(k,k)
X, Y = np.meshgrid(range(k), range(k))
%matplotlib tk
fig = plt.figure()
ax = fig.gca(projection='3d')
ax = fig.gca()
ax.plot_surface(X, Y, T, cmap=cm.hot)

### First with Jacobi

In [ ]:
# Try this for value of max_iters from 10 to 10000, and visualize them with the cell above

t, resvec = Jsolve(A, b, max_iters = 10000)

# Re-plot
T = t.reshape(k,k)
X, Y = np.meshgrid(range(k), range(k))
%matplotlib tk
fig = plt.figure()
ax = fig.gca(projection='3d')
ax = fig.gca()
ax.plot_surface(X, Y, T, cmap=cm.hot)

### Next with CG

In [ ]:
# Try this for value of max_iters from 10 to 10000, and visualize them with the cell above

t, resvec = CGsolve(A, b, max_iters = 10000)

# Re-plot
T = t.reshape(k,k)
X, Y = np.meshgrid(range(k), range(k))
%matplotlib tk
fig = plt.figure()
ax = fig.gca(projection='3d')
ax = fig.gca()
ax.plot_surface(X, Y, T, cmap=cm.hot)

### Comparing Jacobi vs CG

In [ ]:
# Plot relative residual (y axis) versus iteration number (x axis) for both Jacobi and CG

%matplotlib inline
plt.figure()

(xJ,resvecJ) = Jsolve(A, b, tol = 1e-6, max_iters = 1000)
print('\nJacobi iters:', len(resvecJ)-1)
print('last rel res:', resvecJ[-1])
print('computed rel res:', npla.norm(A@xJ - b) / npla.norm(b))
plt.semilogy(resvecJ, label = 'Jacobi')

(xCG,resvecCG) = CGsolve(A, b, tol = 1e-6, max_iters = 1000)
print('\nCG iters:', len(resvecCG)-1)
print('last rel res:', resvecCG[-1])
print('computed rel res:', npla.norm(A@xCG - b) / npla.norm(b))
plt.semilogy(resvecCG, label = 'CG')

plt.legend()
plt.xlabel('iterations')
plt.ylabel('relative residual')
plt.title('Iterative methods for temperature problem with n = %d' % A.shape[0])